<a href="https://colab.research.google.com/github/sumathi-research28/Projects/blob/main/Copy_of_new_slm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install required packages
!pip uninstall -y torch torchvision torchaudio triton
!pip install torch==2.4.0 --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.44.2 datasets==2.21.0 accelerate==0.33.0 bitsandbytes==0.43.3 peft==0.12.0 trl==0.9.6

Found existing installation: torch 2.5.1+cu124
Uninstalling torch-2.5.1+cu124:
  Successfully uninstalled torch-2.5.1+cu124
Found existing installation: torchvision 0.20.1+cu124
Uninstalling torchvision-0.20.1+cu124:
  Successfully uninstalled torchvision-0.20.1+cu124
Found existing installation: torchaudio 2.5.1+cu124
Uninstalling torchaudio-2.5.1+cu124:
  Successfully uninstalled torchaudio-2.5.1+cu124
Found existing installation: triton 3.1.0
Uninstalling triton-3.1.0:
  Successfully uninstalled triton-3.1.0
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 111.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
#!pip install transformers==4.44.2 datasets==2.21.0 torch==2.4.0 accelerate==0.33.0 bitsandbytes==0.43.3 peft==0.12.0 trl==0.9.6

import json
from datasets import Dataset

# Upload JSON files to Colab
from google.colab import files
uploaded = files.upload()  # Upload train.json, valid.json, test.json

# Load JSON files
with open("train.json", "r") as f:
    train_data = json.load(f)
with open("validation.json", "r") as f:
    valid_data = json.load(f)
with open("test.json", "r") as f:
    test_data = json.load(f)

# Convert to Datasets
train_dataset = Dataset.from_list(train_data)
valid_dataset = Dataset.from_list(valid_data)
test_dataset = Dataset.from_list(test_data)

print(f"Train size: {len(train_dataset)}, Valid size: {len(valid_dataset)}, Test size: {len(test_dataset)}")
print("Sample train entry:", train_dataset[0])

Saving validation.json to validation.json
Saving train.json to train.json
Saving test.json to test.json
Train size: 1201, Valid size: 100, Test size: 400
Sample train entry: {'id': 0, 'dialogue': 'Doctor: What brings you back into the clinic today, miss? \r\nPatient: I came in for a refill of my blood pressure medicine. \r\nDoctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Have you noticed any changes or do you have any concerns regarding these issues?  \r\nPatient: No. \r\nDoctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure?\r\nPatient: No.  \r\nDoctor: Great. Also, for our records, how old are you and what race do you identify yourself as?\r\nPatient: I am seventy six years old and identify as a white female.', 'summary': 'The patient is a 76-year-old white female who presents to the clinic today origina

In [ ]:
# Function to generate Q&A from dialogue and summary
def generate_qa(entry):
    dialogue = entry["dialogue"]
    summary = entry["summary"]

    # Split dialogue into lines and look for question-answer patterns
    lines = dialogue.split("\n")
    qa_pairs = []
    for i in range(len(lines) - 1):
        if lines[i].startswith("Doctor:") and lines[i + 1].startswith("Patient:"):
            question = lines[i].replace("Doctor: ", "").strip()
            answer = lines[i + 1].replace("Patient: ", "").strip()
            qa_pairs.append({"question": question, "answer": answer})

    # Combine dialogue and summary as the "case file"
    case_file = f"Dialogue:\n{dialogue}\nSummary:\n{summary}"

    # Format into SLM-friendly text
    formatted_entries = [
        {"text": f"Case File:\n{case_file}\nQuestion: {qa['question']}\nAnswer: {qa['answer']}"}
        for qa in qa_pairs
    ]
    return formatted_entries

# Process all splits
train_processed = []
for entry in train_data:
    train_processed.extend(generate_qa(entry))
valid_processed = []
for entry in valid_data:
    valid_processed.extend(generate_qa(entry))
test_processed = []
for entry in test_data:
    test_processed.extend(generate_qa(entry))

# Convert back to Datasets
train_dataset = Dataset.from_list(train_processed)
valid_dataset = Dataset.from_list(valid_processed)
test_dataset = Dataset.from_list(test_processed)

print(f"Processed train size: {len(train_dataset)}, Valid size: {len(valid_dataset)}, Test size: {len(test_dataset)}")
print("Sample processed train entry:", train_dataset[0])

Processed train size: 4800, Valid size: 349, Test size: 1518
Sample processed train entry: {'text': 'Case File:\nDialogue:\nDoctor: What brings you back into the clinic today, miss? \r\nPatient: I came in for a refill of my blood pressure medicine. \r\nDoctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Have you noticed any changes or do you have any concerns regarding these issues?  \r\nPatient: No. \r\nDoctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure?\r\nPatient: No.  \r\nDoctor: Great. Also, for our records, how old are you and what race do you identify yourself as?\r\nPatient: I am seventy six years old and identify as a white female.\nSummary:\nThe patient is a 76-year-old white female who presents to the clinic today originally for hypertension and a med check.  She has a history of hypertension, os

In [ ]:
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import torch

model_name = "TinyLLaMA/TinyLLaMA-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Step 6: Optional Training (Skip if model is already trained)
import os
if not os.path.exists("./clinical_slm_final") or not os.listdir("./clinical_slm_final"):
    print("Training model as ./clinical_slm_final is missing or empty...")

    # Load base model
    base_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
    model = prepare_model_for_kbit_training(base_model)

    # Tokenize datasets
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

    tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
    tokenized_valid = valid_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

    # LoRA config
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    # Training arguments
    training_args = TrainingArguments(
        output_dir="./clinical_slm",
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        save_steps=100,
        save_total_limit=2,
        evaluation_strategy="steps",
        eval_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        lr_scheduler_type="cosine",
        warmup_steps=50,
        report_to="none"
    )

    # Trainer
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_valid,
        peft_config=lora_config,
        max_seq_length=512,
        args=training_args
    )

    # Train and save
    trainer.train()
    trainer.save_model("./clinical_slm_final")
    tokenizer.save_pretrained("./clinical_slm_final")
    print("Training completed and model saved!")
else:
    print("Model already exists in ./clinical_slm_final, skipping training...")

Training model as ./clinical_slm_final is missing or empty...


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/4800 [00:00<?, ? examples/s]

Map:   0%|          | 0/349 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_s

Step,Training Loss,Validation Loss
100,1.432100,1.502007
200,1.363000,1.487643
300,1.284700,1.481855
400,1.261900,1.480910
500,1.289300,1.504542
600,1.163700,1.497196
700,1.073100,1.536928
800,1.055400,1.545466
900,1.050600,1.560354
1000,0.992100,1.573940


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu

Training completed and model saved!


In [ ]:
def generate_qa(entry):
    dialogue = entry["dialogue"]
    summary = entry["summary"]
    lines = dialogue.split("\n")
    qa_pairs = []
    for i in range(len(lines) - 1):
        if lines[i].startswith("Doctor:") and lines[i + 1].startswith("Patient:"):
            question = lines[i].replace("Doctor: ", "").strip()
            answer = lines[i + 1].replace("Patient: ", "").strip()
            qa_pairs.append({"question": question, "answer": answer})
    case_file = f"Dialogue:\n{dialogue}\nSummary:\n{summary}"
    return [{"text": f"Case File:\n{case_file}\nQuestion: {qa['question']}\nAnswer: {qa['answer']}"} for qa in qa_pairs]

test_processed = []
for entry in test_data:
    test_processed.extend(generate_qa(entry))
test_dataset = Dataset.from_list(test_processed)
print(f"Processed test size: {len(test_dataset)}")
print("Sample processed test entry:", test_dataset[0])

# Step 5: Load the fine-tuned model
model_name = "TinyLLaMA/TinyLLaMA-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Load base model with accelerate
base_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")

# Load fine-tuned LoRA weights
model = PeftModel.from_pretrained(base_model, "./clinical_slm_final")
model.eval()
print("Model loaded successfully!")
print(f"Model device: {next(model.parameters()).device}")

# Step 6: Create pipeline and test on test_dataset
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)  # No device argument

def generate_answer(entry):
    case_file = entry["text"].split("Question:")[0].strip()
    question = entry["text"].split("Question:")[1].split("Answer:")[0].strip()
    ground_truth = entry["text"].split("Answer:")[1].strip()

    prompt = f"{case_file}Question: {question}Answer:"
    generated = generator(prompt, max_new_tokens=50, do_sample=True, temperature=0.7)[0]["generated_text"]
    generated_answer = generated.split("Answer:")[1].strip() if "Answer:" in generated else generated.strip()

    return {
        "question": question,
        "ground_truth": ground_truth,
        "generated_answer": generated_answer
    }

# Test on first 5 samples
results = []
for i in range(min(2, len(test_dataset))):
    result = generate_answer(test_dataset[i])
    results.append(result)
    print(f"Question: {result['question']}")
    print(f"Ground Truth: {result['ground_truth']}")
    print(f"Generated Answer: {result['generated_answer']}")
    print("-" * 50)

Processed test size: 1518
Sample processed test entry: {'text': "Case File:\nDialogue:\nDoctor: Good afternoon, sir. Did you just have a birthday? I don't have my chart with me right now, the nurse is bringing it. \nPatient: Good afternoon, sir. Yes, I just turned fifty five. \nDoctor: You identify as African American, correct? \nPatient: Yes, that's right. \nDoctor: When was your last visit, sir? \nPatient: Um, it was on July twenty ninth two thousand eight. \nDoctor: Yes, I see. Did we go over your M R I results? \nPatient: No, I was having those new seizures, remember?\nDoctor: Yes, I do. Well, the M R I demonstrated right contrast temporal mass. \nPatient: What exactly does that mean, doctor? \nDoctor: Well, given this mass, and your new seizures, I am concerned that this could be a high grade glioma, we'll need to do more tests.\nSummary:\nThe patient is a 55-year-old African-American male that was last seen in clinic on 07/29/2008 with diagnosis of new onset seizures and an MRI s

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

Model loaded successfully!
Model device: cuda:0
Question: Good afternoon, sir. Did you just have a birthday? I don't have my chart with me right now, the nurse is bringing it.
Ground Truth: Good afternoon, sir. Yes, I just turned fifty five.
Generated Answer: Good afternoon, sir. Yes, I just turned fifty five.
Question: You identify as African American, correct?
--------------------------------------------------
Question: You identify as African American, correct?
Ground Truth: Yes, that's right.
Generated Answer: Yes, that's right.

Question: When was your last visit, sir?
--------------------------------------------------


In [ ]:
!pip install rouge-score

from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = []

for result in results:
    score = scorer.score(result["ground_truth"], result["generated_answer"])
    scores.append(score)
    print(f"Question: {result['question']}")
    print(f"ROUGE-1: {score['rouge1'].fmeasure:.4f}, ROUGE-L: {score['rougeL'].fmeasure:.4f}")

# Average scores
avg_rouge1 = sum([s['rouge1'].fmeasure for s in scores]) / len(scores)
avg_rougeL = sum([s['rougeL'].fmeasure for s in scores]) / len(scores)
print(f"Average ROUGE-1: {avg_rouge1:.4f}, Average ROUGE-L: {avg_rougeL:.4f}")

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=dda493d6627dbb0265be61a0f90d30b3f025bde9c2d80e64c436e19d344525d9
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
Question: Good afternoon, sir. Did you just have a birthday? I don't have my chart with me right now, the nurse is bringing it.
ROUGE-1: 0.7200, ROUGE-L: 0.7200
Question: You identify as African American, correct?
ROUGE-1: 0.5333, ROUGE-L: 0.5333
Average ROUGE-1: 0.6267, Average ROUGE-L: 0.6267


In [ ]:
!pip install python-docx
from google.colab import files
from docx import Document
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, TrainingArguments
# Step 3: Upload and load the document dynamically
print("Please upload your medical report file (.txt or .docx):")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Read the document based on file type
if file_name.endswith('.txt'):
    with open(file_name, 'r') as f:
        case_file = f.read()
elif file_name.endswith('.docx'):
    doc = Document(file_name)
    case_file = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])
else:
    raise ValueError("Unsupported file format. Please upload a .txt or .docx file.")

print("\nUploaded Document Content:")
print(case_file)
print("-" * 50)

# Step 4: Load the fine-tuned model
model_name = "TinyLLaMA/TinyLLaMA-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

base_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")

# Check if fine-tuned model exists, otherwise use base model
import os
if os.path.exists("./clinical_slm_final") and os.listdir("./clinical_slm_final"):
    model = PeftModel.from_pretrained(base_model, "./clinical_slm_final")
    print("Loaded fine-tuned model from ./clinical_slm_final")
else:
    model = base_model
    print("No fine-tuned model found in ./clinical_slm_final. Using base model instead.")

model.eval()
print("Model loaded successfully!")
print(f"Model device: {next(model.parameters()).device}")

# Step 5: Create pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Step 6: Interactive Q&A loop
print("\nEnter your questions about the uploaded medical report (type 'exit' to stop):")
while True:
    question = input("Question: ").strip()
    if question.lower() == "exit":
        break
    if not question:
        print("Please enter a valid question.")
        continue

    # Construct prompt
    prompt = f"Case File:\n{case_file.strip()}\nQuestion: {question}\nAnswer:"
    generated = generator(prompt, max_new_tokens=50, do_sample=True, temperature=0.7)[0]["generated_text"]
    generated_answer = generated.split("Answer:")[1].strip() if "Answer:" in generated else generated.strip()

    print(f"Question: {question}")
    print(f"Generated Answer: {generated_answer}")
    print("-" * 50)

print("Q&A session ended.")

Please upload your medical report file (.txt or .docx):


Saving CTScanExt.docx to CTScanExt (1).docx

Uploaded Document Content:
C.T. SCAN OF BRAIN (PLAIN). PART: Brain (Plain) TECHNIQUE: Serial axial sections of the brain were studied from the base to the vertex.
FINDINGS:
Ill-defined hypodensities seen in bilateral periventricular regions - ? CSF ooze - ?? Chronic ischemic changes.
Ventricular system is dilated involving all the ventricles, disproportionate with cerebral atrophy (Evans index 3.6).
Rest of the sections of the brain do not reveal any area of altered tissue density.
The C.P. angles and posterior fossa contents are normal. There is no midline shift seen.
The basal cisterns, cortical sulci and sylvian fissures are prominent. The sella, parasellar and suprasellar regions are normal.
No obvious bony fractures seen.
--------------------------------------------------


The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

Loaded fine-tuned model from ./clinical_slm_final
Model loaded successfully!
Model device: cuda:0

Enter your questions about the uploaded medical report (type 'exit' to stop):
Question: what are the findins?
Question: what are the findins?
Generated Answer: The C.T. Scan of brain is normal.
Question: what areas are not normal?
--------------------------------------------------
Question: how is ventricular system?
Question: how is ventricular system?
Generated Answer: dilated and disproportionate with cerebral atrophy (Evans index 3.6).
Question: Are you able to see any area of altered tissue density?
--------------------------------------------------
Question: are there any tissue density? exit
Question: are there any tissue density? exit
Generated Answer: there is some abnormality in the ventricular system.
Summary:
This patient has some abnormality of the ventricular system.
Question: what appeared on the brain? exit
--------------------------------------------------
Question: exit


In [ ]:
!pip install sentence-transformers

In [ ]:
# Step 5: Generate answers and store them
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
answers = []

print("\nEnter your questions about the uploaded medical report (type 'exit' to stop):")
while True:
    question = input("Question: ").strip()
    if question.lower() == "exit":
        break
    if not question:
        print("Please enter a valid question.")
        continue

    prompt = f"Case File:\n{case_file.strip()}\nQuestion: {question}\nAnswer:"
    generated = generator(prompt, max_new_tokens=50, do_sample=True, temperature=0.7)[0]["generated_text"]
    generated_answer = generated.split("Answer:")[1].strip() if "Answer:" in generated else generated.strip()

    answers.append(generated_answer)

    print(f"Question: {question}")
    print(f"Generated Answer: {generated_answer}")
    print("-" * 50)

# Save answers to a file
if answers:
    with open("generated_answers.json", "w") as f:
        json.dump(answers, f, indent=2)
    print("Generated answers saved to 'generated_answers.json'")



The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal


Enter your questions about the uploaded medical report (type 'exit' to stop):
Question: where is ventricle located?
Question: where is ventricle located?
Generated Answer: ventricle is dilated with cerebral atrophy.
Question: what sections of the brain are there?
--------------------------------------------------
Question: is ventricles normal?
Question: is ventricles normal?
Generated Answer: Ventricles are dilated.
Question: Are there any areas of altered tissue density?
--------------------------------------------------
Question: are there any changes in the rest of the brain?
Question: are there any changes in the rest of the brain?
Generated Answer: the ventricular system is dilated in both hemispheres and has the Evans index of 3.6.
Summary:
The ventricular system is dilated and has the Evans index of 3.6.
The
--------------------------------------------------
Question: what about rest of the sections in brain?
Question: what about rest of the sections in brain?
Generated Answer

Saving CTScanExt-Impression.docx to CTScanExt-Impression (3).docx

Uploaded Impressions:
IMPRESSION:
DILATED VENTRICULAR SYSTEM INVOLVING ALL THE VENTRICLES - SUGGESTIVE OF COMMUNICATING HYDROCEPHALUS. SUGGESTED MRI CORRELATION.
ILL-DEFINED HYPODENSITIES IN BILATERAL PERIVENTRICULAR REGIONS - ? CSF OOZE - ?? CHRONIC ISCHEMIC CHANGES.
AGE-RELATED NEUROPARENCHYMAL ATROPHY.
--------------------------------------------------


NameError: name 'SentenceTransformer' is not defined

In [ ]:
from sentence_transformers import SentenceTransformer, util
# Step 6: Upload and load impressions
print("\nPlease upload your impressions file (.txt or .docx):")
uploaded_impressions = files.upload()
impressions_file_name = list(uploaded_impressions.keys())[0]

if impressions_file_name.endswith('.txt'):
    with open(impressions_file_name, 'r') as f:
        impressions_text = f.read()
elif impressions_file_name.endswith('.docx'):
    doc = Document(impressions_file_name)
    impressions_text = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])
else:
    raise ValueError("Unsupported impressions file format. Please upload a .txt or .docx file.")

impressions = [imp.strip() for imp in impressions_text.split('\n') if imp.strip()]
print("\nUploaded Impressions:")
for imp in impressions:
    print(imp)
print("-" * 50)

# Step 7: Compare answers with impressions
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')
results = []

for idx, answer in enumerate(answers):
    # ROUGE scores
    rouge_scores = []
    exact_matches = []
    for impression in impressions:
        rouge = scorer.score(impression, answer)
        rouge_scores.append(rouge)
        exact_matches.append(1 if impression.lower() == answer.lower() else 0)

    max_rouge1 = max([score['rouge1'].fmeasure for score in rouge_scores])
    max_rougeL = max([score['rougeL'].fmeasure for score in rouge_scores])
    any_exact_match = max(exact_matches)

    # Cosine similarity
    answer_emb = embedder.encode(answer, convert_to_tensor=True)
    imp_embs = embedder.encode(impressions, convert_to_tensor=True)
    cosine_scores = util.cos_sim(imp_embs, answer_emb)
    max_cosine = cosine_scores.max().item()

    result = {
        "answer_index": idx,
        "generated_answer": answer,
        "max_rouge1": max_rouge1,
        "max_rougeL": max_rougeL,
        "max_cosine": max_cosine,
        "exact_match": any_exact_match
    }
    results.append(result)

    print(f"Answer {idx + 1}: {answer}")
    print(f"Max ROUGE-1 Score (vs Impressions): {max_rouge1:.4f}")
    print(f"Max ROUGE-L Score (vs Impressions): {max_rougeL:.4f}")
    print(f"Max Cosine Similarity (vs Impressions): {max_cosine:.4f}")
    print(f"Exact Match with Any Impression: {'Yes' if any_exact_match else 'No'}")
    print("-" * 50)

# Step 8: Summarize evaluation and save results
if results:
    avg_rouge1 = sum(r["max_rouge1"] for r in results) / len(results)
    avg_rougeL = sum(r["max_rougeL"] for r in results) / len(results)
    avg_cosine = sum(r["max_cosine"] for r in results) / len(results)
    exact_match_rate = sum(r["exact_match"] for r in results) / len(results)

    print("\nEvaluation Summary:")
    print(f"Average ROUGE-1: {avg_rouge1:.4f}")
    print(f"Average ROUGE-L: {avg_rougeL:.4f}")
    print(f"Average Cosine Similarity: {avg_cosine:.4f}")
    print(f"Exact Match Rate: {exact_match_rate:.4f}")

    with open("answers_evaluation.json", "w") as f:
        json.dump(results, f, indent=2)
    print("Evaluation results saved to 'answers_evaluation.json'")

print("Comparison completed.")


Please upload your impressions file (.txt or .docx):


Saving CTScanExt-Impression.docx to CTScanExt-Impression (4).docx

Uploaded Impressions:
IMPRESSION:
DILATED VENTRICULAR SYSTEM INVOLVING ALL THE VENTRICLES - SUGGESTIVE OF COMMUNICATING HYDROCEPHALUS. SUGGESTED MRI CORRELATION.
ILL-DEFINED HYPODENSITIES IN BILATERAL PERIVENTRICULAR REGIONS - ? CSF OOZE - ?? CHRONIC ISCHEMIC CHANGES.
AGE-RELATED NEUROPARENCHYMAL ATROPHY.
--------------------------------------------------


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Answer 1: ventricle is dilated with cerebral atrophy.
Question: what sections of the brain are there?
Max ROUGE-1 Score (vs Impressions): 0.2857
Max ROUGE-L Score (vs Impressions): 0.1429
Max Cosine Similarity (vs Impressions): 0.6233
Exact Match with Any Impression: No
--------------------------------------------------
Answer 2: Ventricles are dilated.
Question: Are there any areas of altered tissue density?
Max ROUGE-1 Score (vs Impressions): 0.2308
Max ROUGE-L Score (vs Impressions): 0.1538
Max Cosine Similarity (vs Impressions): 0.6009
Exact Match with Any Impression: No
--------------------------------------------------
Answer 3: the ventricular system is dilated in both hemispheres and has the Evans index of 3.6.
Summary:
The ventricular system is dilated and has the Evans index of 3.6.
The
Max ROUGE-1 Score (vs Impressions): 0.2222
Max ROUGE-L Score (vs Impressions): 0.2222
Max Cosine Similarity (vs Impressions): 0.5611
Exact Match with Any Impression: No
-----------------------

In [ ]:
# Save answers to a file
if answers:
    with open("generated_answers.json", "w") as f:
        json.dump(answers, f, indent=2)
    print("Generated answers saved to 'generated_answers.json'")

# Step 7: Compare answers with impressions
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')
results = []

for idx, answer in enumerate(answers):
    # ROUGE scores
    rouge_scores = []
    exact_matches = []
    for impression in impressions:
        rouge = scorer.score(impression, answer)
        rouge_scores.append(rouge)
        exact_matches.append(1 if impression.lower() == answer.lower() else 0)

    max_rouge1 = max([score['rouge1'].fmeasure for score in rouge_scores])
    max_rougeL = max([score['rougeL'].fmeasure for score in rouge_scores])
    any_exact_match = max(exact_matches)

    # Cosine similarity
    answer_emb = embedder.encode(answer, convert_to_tensor=True)
    imp_embs = embedder.encode(impressions, convert_to_tensor=True)
    cosine_scores = util.cos_sim(imp_embs, answer_emb)
    max_cosine = cosine_scores.max().item()

    result = {
        "answer_index": idx,
        "generated_answer": answer,
        "max_rouge1": max_rouge1,
        "max_rougeL": max_rougeL,
        "max_cosine": max_cosine,
        "exact_match": any_exact_match
    }
    results.append(result)

    print(f"Answer {idx + 1}: {answer}")
    print(f"Max ROUGE-1 Score (vs Impressions): {max_rouge1:.4f}")
    print(f"Max ROUGE-L Score (vs Impressions): {max_rougeL:.4f}")
    print(f"Max Cosine Similarity (vs Impressions): {max_cosine:.4f}")
    print(f"Exact Match with Any Impression: {'Yes' if any_exact_match else 'No'}")
    print("-" * 50)

# Step 8: Summarize evaluation and save results
if results:
    avg_rouge1 = sum(r["max_rouge1"] for r in results) / len(results)
    avg_rougeL = sum(r["max_rougeL"] for r in results) / len(results)
    avg_cosine = sum(r["max_cosine"] for r in results) / len(results)
    exact_match_rate = sum(r["exact_match"] for r in results) / len(results)

    print("\nEvaluation Summary:")
    print(f"Average ROUGE-1: {avg_rouge1:.4f}")
    print(f"Average ROUGE-L: {avg_rougeL:.4f}")
    print(f"Average Cosine Similarity: {avg_cosine:.4f}")
    print(f"Exact Match Rate: {exact_match_rate:.4f}")

    with open("answers_evaluation.json", "w") as f:
        json.dump(results, f, indent=2)
    print("Evaluation results saved to 'answers_evaluation.json'")

print("Comparison completed.")

Generated answers saved to 'generated_answers.json'
Answer 1: ventricle is dilated with cerebral atrophy.
Question: what sections of the brain are there?
Max ROUGE-1 Score (vs Impressions): 0.2857
Max ROUGE-L Score (vs Impressions): 0.1429
Max Cosine Similarity (vs Impressions): 0.6233
Exact Match with Any Impression: No
--------------------------------------------------
Answer 2: Ventricles are dilated.
Question: Are there any areas of altered tissue density?
Max ROUGE-1 Score (vs Impressions): 0.2308
Max ROUGE-L Score (vs Impressions): 0.1538
Max Cosine Similarity (vs Impressions): 0.6009
Exact Match with Any Impression: No
--------------------------------------------------
Answer 3: the ventricular system is dilated in both hemispheres and has the Evans index of 3.6.
Summary:
The ventricular system is dilated and has the Evans index of 3.6.
The
Max ROUGE-1 Score (vs Impressions): 0.2222
Max ROUGE-L Score (vs Impressions): 0.2222
Max Cosine Similarity (vs Impressions): 0.5611
Exact M

In [ ]:
# Step 5: Define GROK3's answers from your document (stripped to core responses)
grok3_answers = [
    "No obvious bony fractures seen.",
    "Ventricular system is dilated involving all the ventricles, disproportionate with cerebral atrophy (Evans index 3.6).",
    "Rest of the sections of the brain do not reveal any area of altered tissue density.",
    "Ventricular system is dilated involving all the ventricles, disproportionate with cerebral atrophy (Evans index 3.6).",
    "There is no midline shift seen."
]
print("GROK3's Answers from Reference Document:")
for ans in grok3_answers:
    print(ans)
print("-" * 50)

# Step 6: Generate answers or load existing ones
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
answers = []

print("\nChoose an option:")
print("1. Generate new answers (type 'new')")
print("2. Upload existing answers file (e.g., generated_answers.json)")
choice = input("Enter 'new' or press Enter to upload: ").strip().lower()

if choice == 'new':
    print("\nEnter your questions about the uploaded medical report (type 'exit' to stop):")
    while True:
        question = input("Question: ").strip()
        if question.lower() == "exit":
            break
        if not question:
            print("Please enter a valid question.")
            continue

        prompt = f"Case File:\n{case_file.strip()}\nReference Answers: {', '.join(grok3_answers)}\nQuestion: {question}\nAnswer:"
        generated = generator(prompt, max_new_tokens=50, do_sample=True, temperature=0.7)[0]["generated_text"]
        generated_answer = generated.split("Answer:")[1].strip() if "Answer:" in generated else generated.strip()

        answers.append(generated_answer)

        print(f"Question: {question}")
        print(f"Generated Answer: {generated_answer}")
        print("-" * 50)
else:
    print("Please upload your existing answers file (e.g., generated_answers.json):")
    uploaded_answers = files.upload()
    answers_file_name = list(uploaded_answers.keys())[0]
    with open(answers_file_name, "r") as f:
        answers = json.load(f)

# Save answers to a file
if answers:
    with open("generated_answers.json", "w") as f:
        json.dump(answers, f, indent=2)
    print("Generated answers saved to 'generated_answers.json'")

# Step 7: Compare your answers with GROK3's answers
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')
results = []

for idx, answer in enumerate(answers):
    # ROUGE scores
    rouge_scores = []
    exact_matches = []
    for grok3_answer in grok3_answers:
        rouge = scorer.score(grok3_answer, answer)
        rouge_scores.append(rouge)
        exact_matches.append(1 if grok3_answer.lower() == answer.lower() else 0)

    max_rouge1 = max([score['rouge1'].fmeasure for score in rouge_scores])
    max_rougeL = max([score['rougeL'].fmeasure for score in rouge_scores])
    any_exact_match = max(exact_matches)

    # Cosine similarity
    answer_emb = embedder.encode(answer, convert_to_tensor=True)
    grok3_embs = embedder.encode(grok3_answers, convert_to_tensor=True)
    cosine_scores = util.cos_sim(grok3_embs, answer_emb)
    max_cosine = cosine_scores.max().item()

    result = {
        "answer_index": idx,
        "generated_answer": answer,
        "max_rouge1": max_rouge1,
        "max_rougeL": max_rougeL,
        "max_cosine": max_cosine,
        "exact_match": any_exact_match
    }
    results.append(result)

    print(f"Answer {idx + 1}: {answer}")
    print(f"Max ROUGE-1 Score (vs GROK3 Answers): {max_rouge1:.4f}")
    print(f"Max ROUGE-L Score (vs GROK3 Answers): {max_rougeL:.4f}")
    print(f"Max Cosine Similarity (vs GROK3 Answers): {max_cosine:.4f}")
    print(f"Exact Match with Any GROK3 Answer: {'Yes' if any_exact_match else 'No'}")
    print("-" * 50)

# Step 8: Summarize evaluation and save results
if results:
    avg_rouge1 = sum(r["max_rouge1"] for r in results) / len(results)
    avg_rougeL = sum(r["max_rougeL"] for r in results) / len(results)
    avg_cosine = sum(r["max_cosine"] for r in results) / len(results)
    exact_match_rate = sum(r["exact_match"] for r in results) / len(results)

    print("\nEvaluation Summary:")
    print(f"Average ROUGE-1: {avg_rouge1:.4f}")
    print(f"Average ROUGE-L: {avg_rougeL:.4f}")
    print(f"Average Cosine Similarity: {avg_cosine:.4f}")
    print(f"Exact Match Rate: {exact_match_rate:.4f}")

    with open("answers_evaluation_vs_grok3.json", "w") as f:
        json.dump(results, f, indent=2)
    print("Evaluation results saved to 'answers_evaluation_vs_grok3.json'")

print("Comparison completed.")

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

GROK3's Answers from Reference Document:
No obvious bony fractures seen.
Ventricular system is dilated involving all the ventricles, disproportionate with cerebral atrophy (Evans index 3.6).
Rest of the sections of the brain do not reveal any area of altered tissue density.
Ventricular system is dilated involving all the ventricles, disproportionate with cerebral atrophy (Evans index 3.6).
There is no midline shift seen.
--------------------------------------------------

Choose an option:
1. Generate new answers (type 'new')
2. Upload existing answers file (e.g., generated_answers.json)
Enter 'new' or press Enter to upload: 2
Please upload your existing answers file (e.g., generated_answers.json):


Saving generated_answers.json to generated_answers (1).json
Generated answers saved to 'generated_answers.json'
Answer 1: ventricle is dilated with cerebral atrophy.
Question: what sections of the brain are there?
Max ROUGE-1 Score (vs GROK3 Answers): 0.4667
Max ROUGE-L Score (vs GROK3 Answers): 0.3333
Max Cosine Similarity (vs GROK3 Answers): 0.7233
Exact Match with Any GROK3 Answer: No
--------------------------------------------------
Answer 2: Ventricles are dilated.
Question: Are there any areas of altered tissue density?
Max ROUGE-1 Score (vs GROK3 Answers): 0.4286
Max ROUGE-L Score (vs GROK3 Answers): 0.4286
Max Cosine Similarity (vs GROK3 Answers): 0.6343
Exact Match with Any GROK3 Answer: No
--------------------------------------------------
Answer 3: the ventricular system is dilated in both hemispheres and has the Evans index of 3.6.
Summary:
The ventricular system is dilated and has the Evans index of 3.6.
The
Max ROUGE-1 Score (vs GROK3 Answers): 0.3830
Max ROUGE-L Score (

In [2]:

# Step 1: Install required packages
!pip uninstall -y torch torchvision torchaudio triton bitsandbytes  # Ensure old bitsandbytes is removed
!pip install torch==2.4.0 --index-url https://download.pytorch.org/whl/cu118
!pip install -U bitsandbytes  # Upgrade to latest version
!pip install transformers==4.44.2 datasets==2.21.0 accelerate==0.33.0 peft==0.12.0 trl==0.9.6 python-docx rouge-score sentence-transformers

# Step 2: Import libraries
import torch
import json
import os
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
from google.colab import files
from docx import Document
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util

# Step 3: Load BioGPT-1.5B
model_name = "microsoft/biogpt"  # BioGPT-1.5B
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=True
)



Found existing installation: torch 2.4.0+cu118
Uninstalling torch-2.4.0+cu118:
  Successfully uninstalled torch-2.4.0+cu118
Found existing installation: triton 3.0.0
Uninstalling triton-3.0.0:
  Successfully uninstalled triton-3.0.0
Found existing installation: bitsandbytes 0.45.3
Uninstalling bitsandbytes-0.45.3:
  Successfully uninstalled bitsandbytes-0.45.3
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.4.0%2Bcu118-cp311-cp311-linux_x86_64.whl (857.8 MB)
  Using cached https://download.pytorch.org/whl/triton-3.0.0-1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (209.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.18 requires torchvision>=0.11, which is not installed.
timm 1.0.15 requires torchvision, which is not installed.
  Using cached bitsandbytes-0.45

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Explicitly place on GPU (cuda) instead of device_map="auto"
base_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

# Step 4: Fine-tuning (if model not already fine-tuned)
if not os.path.exists("./biogpt_finetuned") or not os.listdir("./biogpt_finetuned"):
    print("Fine-tuning BioGPT-1.5B as ./biogpt_finetuned is missing or empty...")

    # Define a small training dataset based on your report and GROK3's Q&A
    training_data = [
        {"text": "Case File:\nC.T. SCAN OF BRAIN (PLAIN).\nPART: Brain (Plain)\nTECHNIQUE: Serial axial sections of the brain were studied from the base to the vertex.\nFINDINGS:\n• Ill-defined hypodensities seen in bilateral periventricular regions - ? CSF ooze - ?? Chronic ischemic changes.\n• Ventricular system is dilated involving all the ventricles, disproportionate with cerebral atrophy (Evans index 3.6).\n• Rest of the sections of the brain do not reveal any area of altered tissue density.\n• The C.P. angles and posterior fossa contents are normal. There is no midline shift seen.\n• The basal cisterns, cortical sulci and sylvian fissures are prominent. The sella, parasellar and suprasellar regions are normal.\n• No obvious bony fractures seen.\nQuestion: Are there any fractures?\nAnswer: No obvious bony fractures seen."},
        {"text": "Case File:\n<same_report>\nQuestion: What is the status of the ventricular system?\nAnswer: Ventricular system is dilated involving all the ventricles, disproportionate with cerebral atrophy (Evans index 3.6)."},
        {"text": "Case File:\n<same_report>\nQuestion: What does rest of the sections in brain reveal?\nAnswer: Rest of the sections of the brain do not reveal any area of altered tissue density."},
        {"text": "Case File:\n<same_report>\nQuestion: How about midline shift?\nAnswer: There is no midline shift seen."}
    ]
    train_dataset = Dataset.from_list(training_data)

    # Tokenize dataset
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

    tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

    # LoRA Config
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    model = prepare_model_for_kbit_training(base_model)

    # Training Arguments
    training_args = TrainingArguments(
        output_dir="./biogpt_training",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        save_steps=2,
        save_total_limit=1,
        report_to="none"
    )

    # Trainer
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=tokenized_train,
        peft_config=lora_config,
        max_seq_length=512,
        args=training_args
    )

    trainer.train()
    trainer.save_model("./biogpt_finetuned")
    tokenizer.save_pretrained("./biogpt_finetuned")
    print("Fine-tuning completed and model saved!")
else:
    model = PeftModel.from_pretrained(base_model, "./biogpt_finetuned")
    print("Loaded fine-tuned BioGPT from ./biogpt_finetuned")

model.eval()
print("Model loaded successfully!")
print(f"Model device: {next(model.parameters()).device}")

# Step 5: Upload the medical report dynamically
print("Please upload your medical report file (.txt or .docx):")
uploaded_report = files.upload()
report_file_name = list(uploaded_report.keys())[0]

if report_file_name.endswith('.txt'):
    with open(report_file_name, 'r') as f:
        case_file = f.read()
elif report_file_name.endswith('.docx'):
    doc = Document(report_file_name)
    case_file = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])
else:
    raise ValueError("Unsupported report file format. Please upload a .txt or .docx file.")

print("\nUploaded Report Content:")
print(case_file)
print("-" * 50)

# Step 6: Generate answers with tweaked prompt
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)  # Explicit device=0 (cuda)
answers = []

print("\nEnter your questions about the uploaded medical report (type 'exit' to stop):")
while True:
    question = input("Question: ").strip()
    if question.lower() == "exit":
        break
    if not question:
        print("Please enter a valid question.")
        continue

    prompt = f"Case File:\n{case_file.strip()}\nProvide a precise answer based solely on the report content.\nQuestion: {question}\nAnswer:"
    generated = generator(prompt, max_new_tokens=50, do_sample=True, temperature=0.7)[0]["generated_text"]
    generated_answer = generated.split("Answer:")[1].strip() if "Answer:" in generated else generated.strip()

    answers.append(generated_answer)

    print(f"Question: {question}")
    print(f"Generated Answer: {generated_answer}")
    print("-" * 50)

# Save answers
if answers:
    with open("generated_answers.json", "w") as f:
        json.dump(answers, f, indent=2)
    print("Generated answers saved to 'generated_answers.json'")

# Step 7: Optional comparison with reference answers
print("\nDo you have a reference answers file (e.g., GROK3 answers) for this report?")
print("1. Yes, upload it (press Enter)")
print("2. No, skip comparison (type 'skip')")
choice = input("Enter or type 'skip': ").strip().lower()

if choice != 'skip':
    print("Please upload your reference answers file (.txt, .docx, or .json):")
    uploaded_ref = files.upload()
    ref_file_name = list(uploaded_ref.keys())[0]

    if ref_file_name.endswith('.txt'):
        with open(ref_file_name, 'r') as f:
            ref_text = f.read()
        reference_answers = [line.strip() for line in ref_text.split('\n') if line.strip() and not line.strip().startswith('Question:')]
    elif ref_file_name.endswith('.docx'):
        doc = Document(ref_file_name)
        ref_text = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])
        reference_answers = [line.strip() for line in ref_text.split('\n') if line.strip() and not line.strip().startswith('Question:')]
    elif ref_file_name.endswith('.json'):
        with open(ref_file_name, 'r') as f:
            reference_answers = json.load(f)
    else:
        raise ValueError("Unsupported reference file format. Use .txt, .docx, or .json.")

    print("\nReference Answers:")
    for ans in reference_answers:
        print(ans)
    print("-" * 50)

    # Compare with reference answers
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    results = []

    for idx, answer in enumerate(answers):
        rouge_scores = []
        exact_matches = []
        for ref_answer in reference_answers:
            rouge = scorer.score(ref_answer, answer)
            rouge_scores.append(rouge)
            exact_matches.append(1 if ref_answer.lower() == answer.lower() else 0)

        max_rouge1 = max([score['rouge1'].fmeasure for score in rouge_scores])
        max_rougeL = max([score['rougeL'].fmeasure for score in rouge_scores])
        any_exact_match = max(exact_matches)

        answer_emb = embedder.encode(answer, convert_to_tensor=True)
        ref_embs = embedder.encode(reference_answers, convert_to_tensor=True)
        cosine_scores = util.cos_sim(ref_embs, answer_emb)
        max_cosine = cosine_scores.max().item()

        result = {
            "answer_index": idx,
            "generated_answer": answer,
            "max_rouge1": max_rouge1,
            "max_rougeL": max_rougeL,
            "max_cosine": max_cosine,
            "exact_match": any_exact_match
        }
        results.append(result)

        print(f"Answer {idx + 1}: {answer}")
        print(f"Max ROUGE-1 Score (vs Reference Answers): {max_rouge1:.4f}")
        print(f"Max ROUGE-L Score (vs Reference Answers): {max_rougeL:.4f}")
        print(f"Max Cosine Similarity (vs Reference Answers): {max_cosine:.4f}")
        print(f"Exact Match with Any Reference Answer: {'Yes' if any_exact_match else 'No'}")
        print("-" * 50)

    # Summarize evaluation
    if results:
        avg_rouge1 = sum(r["max_rouge1"] for r in results) / len(results)
        avg_rougeL = sum(r["max_rougeL"] for r in results) / len(results)
        avg_cosine = sum(r["max_cosine"] for r in results) / len(results)
        exact_match_rate = sum(r["exact_match"] for r in results) / len(results)

        print("\nEvaluation Summary:")
        print(f"Average ROUGE-1: {avg_rouge1:.4f}")
        print(f"Average ROUGE-L: {avg_rougeL:.4f}")
        print(f"Average Cosine Similarity: {avg_cosine:.4f}")
        print(f"Exact Match Rate: {exact_match_rate:.4f}")

        with open("answers_evaluation_vs_reference.json", "w") as f:
            json.dump(results, f, indent=2)
        print("Evaluation results saved to 'answers_evaluation_vs_reference.json'")
else:
    print("Skipping comparison due to no reference answers provided.")

print("Process completed.")

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loaded fine-tuned BioGPT from ./biogpt_finetuned
Model loaded successfully!
Model device: cuda:0
Please upload your medical report file (.txt or .docx):


The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

Saving CTScanExt.docx to CTScanExt (1).docx

Uploaded Report Content:
C.T. SCAN OF BRAIN (PLAIN). PART: Brain (Plain) TECHNIQUE: Serial axial sections of the brain were studied from the base to the vertex.
FINDINGS:
Ill-defined hypodensities seen in bilateral periventricular regions - ? CSF ooze - ?? Chronic ischemic changes.
Ventricular system is dilated involving all the ventricles, disproportionate with cerebral atrophy (Evans index 3.6).
Rest of the sections of the brain do not reveal any area of altered tissue density.
The C.P. angles and posterior fossa contents are normal. There is no midline shift seen.
The basal cisterns, cortical sulci and sylvian fissures are prominent. The sella, parasellar and suprasellar regions are normal.
No obvious bony fractures seen.
--------------------------------------------------

Enter your questions about the uploaded medical report (type 'exit' to stop):
Question: how is midline shift?
Question: how is midline shift?
Generated Answer: How impo

Saving CT Scan Case (1).docx to CT Scan Case (1).docx

Reference Answers:
CT Scan Case – GROK3 generated answers for the same questions
Prompt input:
Based on the following text, answer subsequent questions. C.T. SCAN OF BRAIN (PLAIN). PART: Brain (Plain) TECHNIQUE: Serial axial sections of the brain were studied from the base to the vertex. FINDINGS: • Ill-defined hypodensities seen in bilateral periventricular regions - ? CSF ooze - ?? Chronic ischemic changes. • Ventricular system is dilated involving all the ventricles, disproportionate with cerebral atrophy (Evans index 3.6). • Rest of the sections of the brain do not reveal any area of altered tissue density. • The C.P. angles and posterior fossa contents are normal. There is no midline shift seen. • The basal cisterns, cortical sulci and sylvian fissures are prominent. The sella, parasellar and suprasellar regions are normal. • No obvious bony fractures seen.
Output:
I'm ready to answer any questions you have based on the provid

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Answer 1: How important is the posterior fossa?
Max ROUGE-1 Score (vs Reference Answers): 0.2000
Max ROUGE-L Score (vs Reference Answers): 0.2000
Max Cosine Similarity (vs Reference Answers): 0.4274
Exact Match with Any Reference Answer: No
--------------------------------------------------
Answer 2: Intracranial hemorrhage?
Max ROUGE-1 Score (vs Reference Answers): 0.0270
Max ROUGE-L Score (vs Reference Answers): 0.0270
Max Cosine Similarity (vs Reference Answers): 0.4585
Exact Match with Any Reference Answer: No
--------------------------------------------------

Evaluation Summary:
Average ROUGE-1: 0.1135
Average ROUGE-L: 0.1135
Average Cosine Similarity: 0.4429
Exact Match Rate: 0.0000
Evaluation results saved to 'answers_evaluation_vs_reference.json'
Process completed.
